#### Deep Learning on Dutch Dyslexia Program dataset

Modified from: https://github.com/epodium/EEG_age_prediction

(- Load subject ages from excel file)
- Load raw EEG data from .cnt files
- Apply filters (bandpass)
- Detect potential bad channels and replace them by interpolation
- Detect potential bad epochs and remove them
- Save processed data (and metadata)

#### Imports

In [1]:
import os              
import sys
import glob
import numpy as np  
import pandas as pd 
import mne
from IPython.display import clear_output

# PATH_MAIN = os.path.join(main_path, 'researchdrive', 'ePodium (Projectfolder)')
PATH_MAIN = os.path.join('D:', 'EEG Data', 'DDP Surfdrive')
PATH_METADATA = os.path.join(PATH_MAIN, 'metadata')
PATH_PROCESSED = os.path.join(PATH_MAIN, 'processed')

dataframe = pd.read_excel(os.path.join(PATH_MAIN, 'DDP_age_labels.xlsx'), index_col=0) 

In [2]:
# from eegyolk initialization_functions

def load_dataset(folder_dataset, file_extension = '.bdf', preload=True, max_files_preloaded = 5):
    '''
    This function is for datasets under 5 files. Otherwise use generator_load_dataset
    Reads and returns the files that store the EEG data,
    along with a list of the filenames and paths of these bdf files. 
    Takes as input the top folder location of the dataset.
    '''
    pattern = os.path.join(folder_dataset, '**/*' + file_extension)
    eeg_filepaths = glob.glob(pattern, recursive=True)
    eeg_dataset = []
    eeg_filenames = []
    eeg_filenames_failed_to_load = []

    files_loaded = 0
    files_failed_to_load = 0
    for path in eeg_filepaths:
        filename = os.path.split(path)[1].replace(file_extension, '')

        if(file_extension == '.bdf'):
            raw = mne.io.read_raw_bdf(path,preload=preload)

        if(file_extension == '.cnt'): # .cnt files do not always load.
            try:
                raw = mne.io.read_raw_cnt(path,preload=preload)
            except:
                eeg_filenames_failed_to_load.append(filename)
                files_failed_to_load += 1
                print(f"File {filename} could not be loaded.") 
                continue

        eeg_dataset.append(raw)
        eeg_filenames.append(filename)
        files_loaded += 1
        print(files_loaded, "EEG files loaded")
        if preload and files_loaded >= max_files_preloaded : break

        clear_output(wait=True)
    print(len(eeg_dataset), "EEG files loaded")
    if(files_failed_to_load>0): print(files_failed_to_load, "EEG files failed to load")

    return eeg_dataset, eeg_filenames

#### Load Dataset

In [3]:
# folder_ddp_dataset = os.path.join(data_path, "DDP Dataset") # folder in Surf research drive
main_folder_ddp_dataset = os.path.join("D:", "EEG Data", "DDP Surfdrive") # local folder

ddp_age_folders = ['5mnd mmn', '11mnd mmn', '17mnd mmn', '23mnd mmn',
                    '29mnd mmn', '35mnd mmn', '41mnd mmn', '47mnd mmn']

for i, ddp_age_group in enumerate(ddp_age_folders):
    age_group_folder_location = os.path.join(main_folder_ddp_dataset, ddp_age_folders[i])
    epod_raw, epod_filenames = load_dataset(age_group_folder_location, file_extension = '.cnt', preload=False)
    epod_raw_preload, epod_filenames_preload = load_dataset(age_group_folder_location, file_extension = '.cnt')


Reading 0 ... 464499  =      0.000 ...   928.998 secs...


C:\Users\Floris\AppData\Local\Temp\ipykernel_19784\2451140850.py:26: RuntimeWarning:   Could not parse meas date from the header. Setting to None.
  raw = mne.io.read_raw_cnt(path,preload=preload)
C:\Users\Floris\AppData\Local\Temp\ipykernel_19784\2451140850.py:26: RuntimeWarning: Could not define the number of bytes automatically. Defaulting to 2.
  raw = mne.io.read_raw_cnt(path,preload=preload)


5 EEG files loaded
5 EEG files loaded


#### 

- From https://github.com/epodium/EEG_age_prediction/blob/main/Notebooks/Deep%20learning%20EEG_dataset%20preprocessing_DL.ipynb

In [4]:
def read_cnt_file(file,
                  label_group,
                  event_idx = [2, 3, 4, 5, 12, 13, 14, 15],
                  channel_set = "30",
                  tmin = -0.2,
                  tmax = 0.8,
                  lpass = 0.5, 
                  hpass = 40, 
                  threshold = 5, 
                  max_bad_fraction = 0.2,
                  max_bad_channels = 2):
    """ Function to read cnt file. Run bandpass filter. 
    Then detect and correct/remove bad channels and bad epochs.
    Store resulting epochs as arrays.
    
    Args:
    --------
    file: str
        Name of file to import.
    label_group: int
        Unique ID of specific group (must be >0).
    channel_set: str
        Select among pre-defined channel sets. Here: "30" or "62"
    """
    
    if channel_set == "30":
        channel_set = ['O2', 'O1', 'OZ', 'PZ', 'P4', 'CP4', 'P8', 'C4', 'TP8', 'T8', 'P7', 
                       'P3', 'CP3', 'CPZ', 'CZ', 'FC4', 'FT8', 'TP7', 'C3', 'FCZ', 'FZ', 
                       'F4', 'F8', 'T7', 'FT7', 'FC3', 'F3', 'FP2', 'F7', 'FP1']
    elif channel_set == "62":
        channel_set = ['O2', 'O1', 'OZ', 'PZ', 'P4', 'CP4', 'P8', 'C4', 'TP8', 'T8', 'P7', 
                       'P3', 'CP3', 'CPZ', 'CZ', 'FC4', 'FT8', 'TP7', 'C3', 'FCZ', 'FZ', 
                       'F4', 'F8', 'T7', 'FT7', 'FC3', 'F3', 'FP2', 'F7', 'FP1', 'AFZ', 'PO3', 
                       'P1', 'POZ', 'P2', 'PO4', 'CP2', 'P6', 'M1', 'CP6', 'C6', 'PO8', 'PO7', 
                       'P5', 'CP5', 'CP1', 'C1', 'C2', 'FC2', 'FC6', 'C5', 'FC1', 'F2', 'F6', 
                       'FC5', 'F1', 'AF4', 'AF8', 'F5', 'AF7', 'AF3', 'FPZ']
    else:
        print("Predefined channel set given by 'channel_set' not known...")
        
    
    # Initialize array
    signal_collection = np.zeros((0,len(channel_set),501))
    label_collection = [] #np.zeros((0))
    channel_names_collection = []
    
    # Import file
    try:
        data_raw = mne.io.read_raw_cnt(file, eog='auto', preload=True, verbose=False)
    except ValueError:
        print("ValueError")
        print("Could not load file:", file)
        return None, None, None
    
    # Band-pass filter (between 0.5 and 40 Hz. was 0.5 to 30Hz in Stober 2016)
    data_raw.filter(0.5, 40, fir_design='firwin')

    # Get events from annotations in the data
    events_from_annot, event_dict = mne.events_from_annotations(data_raw)
    
    # Set baseline:
    baseline = (None, 0)  # means from the first instant to t = 0

    # Select channels to exclude (if any)
    channels_exclude = [x for x in data_raw.ch_names if x not in channel_set]
    channels_exclude = [x for x in channels_exclude if x not in ['HEOG', 'VEOG']]
    
    for event_id in event_idx:
        if str(event_id) in event_dict:
            # Pick EEG channels
            picks = mne.pick_types(data_raw.info, meg=False, eeg=True, stim=False, eog=False,
                               #exclude=data_exclude)#'bads'])
                                   include=channel_set, exclude=channels_exclude)#'bads'])

            epochs = mne.Epochs(data_raw, events=events_from_annot, event_id=event_dict,
                                tmin=tmin, tmax=tmax, proj=True, picks=picks,
                                baseline=baseline, preload=True, event_repeated='merge', verbose=False)

            # Detect potential bad channels and epochs
            bad_channels, bad_epochs = select_bad_epochs(epochs,
                                                                          event_id,
                                                                          threshold = threshold,
                                                                          max_bad_fraction = max_bad_fraction)

            # Interpolate bad channels
            # ------------------------------------------------------------------
            if len(bad_channels) > 0:
                if len(bad_channels) > max_bad_channels:
                    print(20*'--')
                    print("Found too many bad channels (" + str(len(bad_channels)) + ")")
                    return None, None, None
                else:
                    montage = mne.channels.make_standard_montage('standard_1020')
                    montage.ch_names = [ch_name.upper() for ch_name in montage.ch_names]
                    data_raw.set_montage(montage)
                    
                    # MARK: Think about using all channels before removing (62 -> 30), to enable for better interpolation
                    
                    # Mark bad channels:
                    data_raw.info['bads'] = bad_channels
                    # Pick EEG channels:
                    picks = mne.pick_types(data_raw.info, meg=False, eeg=True, stim=False, eog=False,
                                       #exclude=data_exclude)#'bads'])
                                       include=channel_set, exclude=channels_exclude)#'bads'])
                    epochs = mne.Epochs(data_raw, events=events_from_annot, event_id=event_dict,
                                        tmin=tmin, tmax=tmax, proj=True, picks=picks,
                                        baseline=baseline, preload=True, verbose=False)
                    
                    # Interpolate bad channels using functionality of 'mne'
                    epochs.interpolate_bads()
                    

            # Get signals as array and add to total collection
            channel_names_collection.append(epochs.ch_names)
            signals_cleaned = epochs[str(event_id)].drop(bad_epochs).get_data()
            signal_collection = np.concatenate((signal_collection, signals_cleaned), axis=0)
            label_collection += [event_id + label_group] * signals_cleaned.shape[0]

    return signal_collection, label_collection, channel_names_collection

In [5]:
"""Functions to import and process EEG data from cnt files.
"""
def standardize_EEG(data_array,
                    std_aim = 1,
                    centering = 'per_channel',
                    scaling = 'global'):
    """ Center data around 0 and adjust standard deviation.

    Args:
    --------
    data_array: np.ndarray
        Input data.
    std_aim: float/int
        Target standard deviation for rescaling/normalization.
    centering: str
        Specify if centering should be done "per_channel", or "global".
    scaling: str
        Specify if scaling should be done "per_channel", or "global".
    """
    if centering == 'global':
        data_mean = data_array.mean()

        # Center around 0
        data_array = data_array - data_mean

    elif centering == 'per_channel':
        for i in range(data_array.shape[1]):

            data_mean = data_array[:,i,:].mean()

            # Center around 0
            data_array[:,i,:] = data_array[:,i,:] - data_mean

    else:
        print("Centering method not known.")
        return None

    if scaling == 'global':
        data_std = data_array.std()

        # Adjust std to std_aim
        data_array = data_array * std_aim/data_std

    elif scaling == 'per_channel':
        for i in range(data_array.shape[1]):

            data_std = data_array[:,i,:].std()

            # Adjust std to std_aim
            data_array[:,i,:] = data_array[:,i,:] * std_aim/data_std
    else:
        print("Given method is not known.")
        return None

    return data_array

#### Helper Function

In [6]:

def select_bad_epochs(epochs, stimuli, threshold = 5, max_bad_fraction = 0.2):
    """
    Function to find suspect epochs and channels --> still might need manual inspection!
    
    Args:
    --------
    epochs: epochs object (mne)
    
    stimuli: int/str
        Stimuli to pick epochs for.         
    threshold: float/int
        Relative threshold. Anything channel with variance > threshold*mean OR < threshold*mean
        will be considered suspect. Default = 5.   
    max_bad_fraction: float
        Maximum fraction of bad epochs. If number is higher for one channel, call it a 'bad' channel
    """
    bad_epochs = set()
    bad_channels = []
    
    from collections import Counter
    
    signals = epochs[str(stimuli)].get_data()
    max_bad_epochs = max_bad_fraction*signals.shape[0]
    
    # Find outliers in episode STD and max-min difference:
    signals_std = np.std(signals, axis=2)
    signals_minmax = np.amax(signals, axis=2) - np.amin(signals, axis=2)
    
    outliers_high = np.where((signals_std > threshold*np.mean(signals_std)) | (signals_minmax > threshold*np.mean(signals_minmax)))
    outliers_low = np.where((signals_std < 1/threshold*np.mean(signals_std)) | (signals_minmax < 1/threshold*np.mean(signals_minmax)))
    outliers = (np.concatenate((outliers_high[0], outliers_low[0])), np.concatenate((outliers_high[1], outliers_low[1])) ) 
    
    if len(outliers[0]) > 0:
        print("Found", len(set(outliers[0])), "bad epochs in a total of", len(set(outliers[1])), " channels.")
        occurences = [(Counter(outliers[1])[x], x) for x in list(Counter(outliers[1]))]
        for occur, channel in occurences:
            if occur > max_bad_epochs:
                print("Found bad channel (more than", max_bad_epochs, " bad epochs): Channel no: ", channel )
                bad_channels.append(channel)
            else:
                # only add bad epochs for non-bad channels
                bad_epochs = bad_epochs|set(outliers[0][outliers[1] == channel])
                
        print("Marked", len(bad_epochs), "bad epochs in a total of", signals.shape[0], " epochs.")
        
#        # Remove bad data:
#        signals = np.delete(signals, bad_channels, 1)
#        signals = np.delete(signals, list(bad_epochs), 0)
        
    else:
        print("No outliers found with given threshold.")
    
    return [epochs.ch_names[x] for x in bad_channels], list(bad_epochs)

In [8]:
import re
from itertools import islice

for i, file in islice(dataframe.iterrows(), 0, None): # Enables you to set starting row of DataFrame
    print(f"Checking out file: {file['cnt_file']}")
        
    filename_processed_raw = os.path.join(PATH_PROCESSED, file['cnt_file'] + ".npy")
    # filename_metadata = os.path.join(PATH_PROCESSED, "processed_metadata_" + file['cnt_file'] + ".csv")
    
    # Step 1: Check if file don't already exist, else skip
    if os.path.exists(filename_processed_raw):
        print(f"Skipping because {filename_processed_raw} already exists.")
        continue
    
    
    # Step 2: Import data and labels 
    signal_collect, label_collect, ch_names = read_cnt_file(file['cnt_path'],
                                                            file['age_days'],
                                                            event_idx = [2, 3, 4, 5, 12, 13, 14, 15],
                                                            channel_set = "30",
                                                            tmin = -0.2,
                                                            tmax = 0.8,
                                                            lpass = 0.5, 
                                                            hpass = 40, 
                                                            threshold = 5, 
                                                            max_bad_fraction = 0.2,
                                                            max_bad_channels = 2)    
    
    if signal_collect is None:
        continue
        
    # Step 3: standardize data
    if signal_collect is not None:
        signal_collect = standardize_EEG(signal_collect,
                                         std_aim = 1,
                                         centering = 'per_channel',
                                         scaling = 'global')

    # Step 4: Save raw data and metadata
    if signal_collect is not None:
        np.save(filename_processed_raw, signal_collect)
        # pd.DataFrame(file).transpose().to_csv(filename_metadata, sep=',', index=False, header=True)     
    
    # TODO: 1. Try saving 2. Retry after n seconds if fails 3. If fails m times, continue

Checking out file: 001_11_jc_mmn36_wk_mmn25_wk_mmn47_wk_mmn58_wk.cnt


C:\Users\Floris\AppData\Local\Temp\ipykernel_19784\1691157752.py:48: RuntimeWarning:   Could not parse meas date from the header. Setting to None.
  data_raw = mne.io.read_raw_cnt(file, eog='auto', preload=True, verbose=False)
C:\Users\Floris\AppData\Local\Temp\ipykernel_19784\1691157752.py:48: RuntimeWarning: Could not define the number of bytes automatically. Defaulting to 2.
  data_raw = mne.io.read_raw_cnt(file, eog='auto', preload=True, verbose=False)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (6.602 sec)

Used Annotations descriptions: ['2', '3', '4', '5', '55', '66', '77', '88']
Found 247 bad epochs in a total of 30  channels.
Found bad channel (more than 72.0  bad epochs): Channel no:  9
Found bad channel (more than 72.0  bad epochs): Channel no:  4
Marked 129 bad epochs in a total of 360  epochs.
Interpolating bad channels
    Automatic origin fit: head of radius 96.2 mm
Computing interpolation matrix from 28 sensor positions
In

C:\Users\Floris\AppData\Local\Temp\ipykernel_19784\1691157752.py:48: RuntimeWarning:   Could not parse meas date from the header. Setting to None.
  data_raw = mne.io.read_raw_cnt(file, eog='auto', preload=True, verbose=False)
C:\Users\Floris\AppData\Local\Temp\ipykernel_19784\1691157752.py:48: RuntimeWarning: Could not define the number of bytes automatically. Defaulting to 2.
  data_raw = mne.io.read_raw_cnt(file, eog='auto', preload=True, verbose=False)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (6.602 sec)

Used Annotations descriptions: ['0', '12', '13', '14', '15', '2', '3', '4', '5', '55', '66', '77', '88']
Found 110 bad epochs in a total of 23  channels.
Marked 110 bad epochs in a total of 400  epochs.
Dropped 110 epochs: 2, 6, 10, 11, 12, 13, 14, 18, 19, 20, 21, 23, 25, 27, 28, 29, 30, 31, 33, 34, 36, 38, 39, 42, 45, 46, 55, 56, 66, 71, 76, 96, 97, 99, 103, 114, 119, 121, 123, 124, 126, 128, 132, 138, 148, 151, 152, 156, 163, 16

...

c:\Floris\Python Folder\Python 3.7\VirtualEnvironments\VENV\lib\site-packages\mne\io\cnt\cnt.py:239: RuntimeWarning: overflow encountered in long_scalars
  data_size // (n_samples * n_channels) not in [2, 4]):
C:\Users\Floris\AppData\Local\Temp\ipykernel_19784\1691157752.py:48: RuntimeWarning: Could not define the number of bytes automatically. Defaulting to 2.
  data_raw = mne.io.read_raw_cnt(file, eog='auto', preload=True, verbose=False)


ValueError
Could not load file: D:EEG Data\DDP Surfdrive\41mnd mmn\333_41_jc_mmn39_wk.cnt
Checking out file: 334_41_mc_mmn39_wk.cnt
Skipping because D:EEG Data\DDP Surfdrive\processed\334_41_mc_mmn39_wk.cnt.npy already exists.
Checking out file: 335_41_mc_mmn39_wk.cnt
Skipping because D:EEG Data\DDP Surfdrive\processed\335_41_mc_mmn39_wk.cnt.npy already exists.
Checking out file: 337_41_jc_mmn36_wk.cnt
Skipping because D:EEG Data\DDP Surfdrive\processed\337_41_jc_mmn36_wk.cnt.npy already exists.
Checking out file: 337_41_jc_mmn39_wk.cnt
Skipping because D:EEG Data\DDP Surfdrive\processed\337_41_jc_mmn39_wk.cnt.npy already exists.
Checking out file: 340_41_mc_mmn39_mix.cnt
Skipping because D:EEG Data\DDP Surfdrive\processed\340_41_mc_mmn39_mix.cnt.npy already exists.
Checking out file: 343_41_mc_mmn39_mix.cnt
Skipping because D:EEG Data\DDP Surfdrive\processed\343_41_mc_mmn39_mix.cnt.npy already exists.
Checking out file: 344_41_jc_mmn39_wk.cnt
Skipping because D:EEG Data\DDP Surfdrive\

C:\Users\Floris\AppData\Local\Temp\ipykernel_19784\1691157752.py:48: RuntimeWarning: Could not define the number of bytes automatically. Defaulting to 2.
  data_raw = mne.io.read_raw_cnt(file, eog='auto', preload=True, verbose=False)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (6.602 sec)

Used Annotations descriptions: ['13', '3', '99']
Found 470 bad epochs in a total of 30  channels.
Found bad channel (more than 120.0  bad epochs): Channel no:  0
Found bad channel (more than 120.0  bad epochs): Channel no:  1
Found bad channel (more than 120.0  bad epochs): Channel no:  2
Found bad channel (more than 120.0  bad epochs): Channel no:  3
Found bad channel (more than 120.0  bad epochs): Channel no:  4
Found bad channe

C:\Users\Floris\AppData\Local\Temp\ipykernel_19784\1691157752.py:48: RuntimeWarning:   Could not parse meas date from the header. Setting to None.
  data_raw = mne.io.read_raw_cnt(file, eog='auto', preload=True, verbose=False)
c:\Floris\Python Folder\Python 3.7\VirtualEnvironments\VENV\lib\site-packages\mne\io\cnt\cnt.py:239: RuntimeWarning: overflow encountered in long_scalars
  data_size // (n_samples * n_channels) not in [2, 4]):
C:\Users\Floris\AppData\Local\Temp\ipykernel_19784\1691157752.py:48: RuntimeWarning: Could not define the number of bytes automatically. Defaulting to 2.
  data_raw = mne.io.read_raw_cnt(file, eog='auto', preload=True, verbose=False)
